In [1]:
import json
import numpy as np
from PIL import Image, ImageDraw
from pathlib import Path
import polygon

In [2]:
class Logger:
    def __init__(self):
        self.on = True
    
    def print(self, text):
        if self.on:
            print(text)

In [10]:
def load_ppocrlabel(path,
                    label_file_name = "Label.txt",
                    deprecated_labels = ["(한자)", "(일본어)"],
                    only_internal_text_label = True,
                    print_log = False
                    ):
    """_summary_

    Args:
        path (str): ppocrlabel 프로그램을 사용해 만든 레이블 파일의 경로
        deprecated_labels (list): 안쓰는 레이블 리스트 (삭제됨)
        only_internal_text_label (bool): 모든 텍스트가 간판 내에만 있는지 확인하고 그렇지 않은 이미지는 제외
        print_log (bool): 진행 상황 로그를 출력할 지 여부
        
    Returns:
        list: 레이블 정보를 로드하여 리스트 형태로 반환
        
        result = [image_label, ...]
        image_label = {"sign":label, "text":[label, ...]}
        label = {"transcription":str , "points": polygon}
        polygon: [(x1, y1), (x2, y2), ... , (xn, yn)]
        
    """

    logger = Logger()
    logger.on = print_log
    
    
    # 데이터 로드
    path = Path(path)/label_file_name
    def load_data(path):
        with open(path) as f:
            lines = [line.rstrip("\n") for line in f.readlines()]
        return lines
    logger.print("Load all data")
    lines = load_data(path)
    logger.print("\n")


    # 각 줄을 샘플로 변환
    def line_to_sample(line):
        image_path, labels = line.split("\t")
        labels = json.loads(labels)
        return {"image_path":image_path, "labels": labels}
    logger.print("Convert text data into sample data")
    samples = [line_to_sample(line) for line in lines]
    logger.print("\n")
    
    # 이미지가 존재하지 않는 샘플이 있는지 확인
    def check_image_exist(samples, print_log = False):
        remove_num = 0
        new_samples = []
        for sample in samples:
            image_path = Path(sample["image_path"])
            if not image_path.exists():
                remove_num += 1
                if print_log:
                    logger.print(f"Removed {sample['image_path']}")
            else:
                new_samples.append(sample)
        return new_samples, remove_num
    logger.print("Check the label has valid image path")
    samples, remove_num = check_image_exist(samples)
    logger.print(f"\tTotal {remove_num} of samples without image was removed\n")
    
    # 간판과 텍스트 레이블 구분
    def devide_sign_and_text(sample, print_log = False):
        sign_labels, text_labels = [], []
        for label in sample["labels"]:
            if label["transcription"] == "@@@":
                sign_labels.append(label)
            else:
                text_labels.append(label)
        return {"image_path":sample["image_path"], "sign_labels":sign_labels, "text_labels":text_labels}
    logger.print("Distingush sign and test label")
    samples = [devide_sign_and_text(sample) for sample in samples]
    logger.print("\n")

    # 모든 텍스트가 간판 안에 있는 이미지 외에 제거
    def remain_only_internal_samples(samples, print_log = False):
        remove_num = 0
        new_samples = []
        for sample in samples:
            if not all([any([polygon.is_polygon_inside_polygon(text_label["points"], sign_label["points"]) for sign_label in sample["sign_labels"]]) for text_label in sample["text_labels"]]):
                remove_num += 1                
                if print_log:
                    logger.print(f"유효하지 않아 제거됨: {sample['image_path']}")
            else:
                new_samples.append(sample)
        return samples, remove_num

    if only_internal_text_label:
        logger.print("Check and remove the image whose all text labels are included in the sign label")
        samples, remove_num = remain_only_internal_samples(samples)
        logger.print(f"\t{remove_num} of images were removed for including one more not internal text lable\n")
        

    # 제거할 텍스트 레이블 제거
    def remove_target_label(samples, print_log = False):
        remove_num = 0
        samples = samples.copy()
        for sample in samples:
            text_labels = []
            for text_label in sample["text_labels"]:
                # 제거할 텍스트 레이블 제거
                if text_label["transcription"] in deprecated_labels:
                    if print_log:
                        logger.print(f"Removed {text_label}")
                    remove_num += 1
                else:
                    text_labels.append(text_label)
            sample["text_labels"] = text_labels
            return samples, remove_num    
        
    if deprecated_labels is not None:
        logger.print("Remove the text labels corresponding the text labels to remove")
        samples, remove_num = remove_target_label(samples)
        logger.print(f"\t{remove_num} of text labels were removed\n")
    
    
    
    def group_sign_and_text_lables(samples):
        samples = samples.copy()
        for sample in samples:
            sign_and_text_labels = []        
            for sign_label in sample["sign_labels"]:
                sign_text_labels = []
                for text_label in sample["text_labels"]:
                    if polygon.is_polygon_inside_polygon(text_label["points"], sign_label["points"]):
                        sign_text_labels.append(text_label)
                
                sign_and_text_labels.append({"sign":sign_label, "text":sign_text_labels}) 
            del sample["sign_labels"]
            del sample["text_labels"]
            sample["labels"] =  sign_and_text_labels
        return samples
    logger.print("Group sign and internal text labels")
    samples = group_sign_and_text_lables(samples)
    print("\n")
    
    logger.print(f"Total image labels: {len(samples)}")
    logger.print(f"Total sign labels: {sum([len(sample['labels']) for sample in samples])}")
    logger.print(f"Total text labels: {sum([sum([len(label['text']) for label in sample['labels']]) for sample in samples])}")
    return samples

def make_and_save_detection_dataset(samples, save_dir, dir_size=1000, label_file_name = "label.txt"):
    assert not save_dir.exists(), f"please remove {save_dir}"
    label_path = save_dir/label_file_name
    image_idx = 0

    for sample in samples:  
        image_path, labels = sample["image_path"], sample["labels"]  
        image = Image.open(image_path)
        for label in labels:
            sign_label = label["sign"]
            text_labels = label["text"]

            cropped_image = polygon.crop_by_polygon(image, sign_label["points"])
            image_path = save_dir/f"{(image_idx//dir_size + 1)}"/f"{image_idx+1}.png"
            image_path.parent.mkdir(parents=True, exist_ok=True)
            cropped_image.save(image_path)
            with open(label_path, "a") as f:
                f.write(f"{str(image_path)}\t{json.dumps(text_labels)}\n")

            image_idx += 1

def make_and_save_recognition_dataset(samples, save_dir, dir_size=1000, label_file_name = "label.txt"):
    assert not save_dir.exists(), f"please remove {save_dir}"
    label_path = save_dir/label_file_name
    image_idx = 0

    for sample in samples:  
        image_path, labels = sample["image_path"], sample["labels"]
         
        image = Image.open(image_path)
        for label in labels:
            sign_label = label["sign"]
            text_labels = label["text"]
            
            for text_label in text_labels:
                cropped_image = polygon.crop_by_polygon(image, text_label["points"])
                image_path = save_dir/f"{(image_idx//dir_size + 1)}"/f"{image_idx+1}.png"
                image_path.parent.mkdir(parents=True, exist_ok=True)
                cropped_image.save(image_path)
                with open(label_path, "a") as f:
                    f.write(f"{str(image_path)}\t{text_label['transcription']}\n")

                image_idx += 1

In [20]:
label_file_path = "./sample"
samples = load_ppocrlabel(label_file_path, print_log=True)

# make_and_save_detection_dataset(samples, Path("./result/det"))
# make_and_save_recognition_dataset(samples, Path("./result/rec"))

Load all data


Convert text data into sample data


Check the label has valid image path
	Total 1 of samples without image was removed

Distingush sign and test label


Check and remove the image whose all text labels are included in the sign label
	0 of images were removed for including one more not internal text lable

Remove the text labels corresponding the text labels to remove
	0 of text labels were removed

Group sign and internal text labels


Total image labels: 2
Total sign labels: 3
Total text labels: 9
